# Existential Risk Estimates Database

This notebook processes data from the existential risk (x-risk) estimates database from [this EA Forum post](https://forum.effectivealtruism.org/posts/JQQAQrunyGGhzE23a/database-of-existential-risk-estimates).

Since the entries in the database vary in format and often use natural language, it is necessary to sanitize them for further analysis.

## Pre-Processing

In this step we fiddle with the dataset to get something more readily usable. Among other things, we:
- Create a risk category column, and get rid of the "headers" for things like AI, or Total Risk, etc.
- Remove columns that aren't necessary for the analysis

In [16]:
# Load the data and extract the relevant columns

import pandas as pd
import numpy as np

# Load excel spreadsheet. Load all sheets.
excel = pd.ExcelFile('./data/xrisk-estimates-database-20241204.xlsx')
# List worksheets
excel.sheet_names

['Overall notes',
 'Existential-risk-level estimate',
 'Conditional existential-risk-le',
 'Estimates of somewhat less extr',
 'Other potential estimates or so']

In [17]:
# Use the 'Existential-risk-level estimate' worksheet.
# the first 5 columns are instructions, so skip them.
df = excel.parse('Existential-risk-level estimate', skiprows=5, index_row=0)
df.head()

,Who is the estimator?,When was the estimate made/published?,What is the estimator estimating?,What is their estimate?,Source,Have I properly read the source myself?,Is this estimate included in Beard et al.'s appendix?,Other notes,Unnamed: 8,Unnamed: 9
0,“Total risk” (or similar),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Toby Ord,2020,“Total existential risk” by 2120,~17% (~1 in 6),The Precipice,Yes,No,"Ord writes: ""Don’t take these numbers to be co...",NaN,NaN
2,GCR Conference,2008,“Overall risk of extinction prior to 2100”,0.19,https://www.fhi.ox.ac.uk/reports/2008-1.pdf,Yes,Yes,This is the median. The report about these est...,NaN,NaN
3,Will MacAskill,2019/2020,Existential risk in the 21st century,0.01,https://80000hours.org/podcast/episodes/will-m...,Yes,No,NaN,NaN,NaN
4,"Ben Todd or 80,000 Hours",2017,Extinction risk “in the next century”,Probably at or above 3%,https://80000hours.org/articles/extinction-risk/,Yes,No,NaN,NaN,NaN


In [18]:
# Print number of columns
print(f"DataFrame has {len(df.columns)} columns")

DataFrame has 10 columns


In [19]:
# Print columns
df.columns

Index(['Who is the estimator? ', 'When was the estimate made/published?',
       'What is the estimator estimating?', 'What is their estimate?',
       'Source', 'Have I properly read the source myself?',
       'Is this estimate included in Beard et al.'s appendix?', 'Other notes',
       'Unnamed: 8', 'Unnamed: 9'],
      dtype='object')

In [20]:
# Rename columns to something more concise
new_column_names = [
  'estimator',
  'date',
  'estimation_measure',
  'estimation',
  'source',
  'source_read_by_estimator',
  'estimate_included_in_beard_et_al',
  'other_notes',
  'unknown_column_1',
'unknown_column_2'
]
df.columns = new_column_names
df.head()

,estimator,date,estimation_measure,estimation,source,source_read_by_estimator,estimate_included_in_beard_et_al,other_notes,unknown_column_1,unknown_column_2
0,“Total risk” (or similar),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Toby Ord,2020,“Total existential risk” by 2120,~17% (~1 in 6),The Precipice,Yes,No,"Ord writes: ""Don’t take these numbers to be co...",NaN,NaN
2,GCR Conference,2008,“Overall risk of extinction prior to 2100”,0.19,https://www.fhi.ox.ac.uk/reports/2008-1.pdf,Yes,Yes,This is the median. The report about these est...,NaN,NaN
3,Will MacAskill,2019/2020,Existential risk in the 21st century,0.01,https://80000hours.org/podcast/episodes/will-m...,Yes,No,NaN,NaN,NaN
4,"Ben Todd or 80,000 Hours",2017,Extinction risk “in the next century”,Probably at or above 3%,https://80000hours.org/articles/extinction-risk/,Yes,No,NaN,NaN,NaN


In [21]:
# Drop unnecessary columns (last 2, which are unknown)
df = df.drop(columns=['unknown_column_1', 'unknown_column_2'])
df.sample(3)

,estimator,date,estimation_measure,estimation,source,source_read_by_estimator,estimate_included_in_beard_et_al,other_notes
90,GCR Conference,2008,Human extinction by 2100 as a result of “wars ...,0.04,https://www.fhi.ox.ac.uk/reports/2008-1.pdf,Yes,Yes,This is the median. Beard et al.'s appendix sa...
84,Toby Ord,2020,“Total natural [existential] risk” by 2120,"~0.01% (~1 in 10,000)",The Precipice,Yes,No,NaN
100,Paul Christiano,2021,Expected fraction of total potential value tha...,0.2,Comment,Yes,No,Response to Wei Dai's question shown in the ab...


In [22]:
# Add empty column for risk category
df['risk_category'] = np.nan
df.sample(3)

,estimator,date,estimation_measure,estimation,source,source_read_by_estimator,estimate_included_in_beard_et_al,other_notes,risk_category
94,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
51,Nanotechnology,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25,Toby Ord,2020,Existential catastrophe by 2120 as a result of...,~10%,The Precipice,Yes,No,Commentary here: https://forum.effectivealtrui...,NaN


In [23]:
# The first row indicates what type of risk is being estimated.
# 
# Under the first row are the total risk estimates. 
# 
# Further down below are estimates for other x-risks.
# 
# The categories are: "Total risk (or similar)", "AI", "Biorisk", "Nanotechnology", "Climate Change", "Natural risks (excluding natural pandemics)", "War", "Explicitly about only unrecoverable dystopia and/or unrecoverable collapse (not extinction)", "Miscellaneous".
risk_categories = [
    '“Total risk” (or similar)',
    'AI',
    'Biorisk',
    'Nanotechnology',
    'Climate Change',
    'Natural risks (excluding natural pandemics)',
    'War',
    'Explicitly about only unrecoverable dystopia and/or unrecoverable collapse (not extinction)',
    'Miscellaneous']

risk_categories_aliases = [
    'total',
    'ai',
    'biorisk',
    'nanotechnology',
    'climate_change',
    'natural_risks',
    'war',
    'dystopia',
    'miscellaneous'
]

# Get rows that contain a risk category in the first column
risk_category_rows = df[df['estimator'].isin(risk_categories)]['estimator']
risk_category_rows 

0                             “Total risk” (or similar)
17                                                   AI
39                                              Biorisk
51                                       Nanotechnology
78          Natural risks (excluding natural pandemics)
87                                                  War
92    Explicitly about only unrecoverable dystopia a...
95                                        Miscellaneous
Name: estimator, dtype: object

In [24]:
# Create a copy of the dataframe
df_with_risk_category = df.copy()

# Initialize the current risk category
current_risk_category = None

# Iterate over the dataframe rows
for i, row in df_with_risk_category.iterrows():
  if row['estimator'] in risk_category_rows.values:
    # Update the current risk category
    current_risk_category = row['estimator']
  # Set the risk category for the current row
  df_with_risk_category.at[i, 'risk_category'] = current_risk_category

df_with_risk_category.sample(3)

/var/folders/gp/134vcy4x6c19634zvbkbff900000gn/T/ipykernel_37345/2712489716.py:13: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '“Total risk” (or similar)' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_with_risk_category.at[i, 'risk_category'] = current_risk_category


,estimator,date,estimation_measure,estimation,source,source_read_by_estimator,estimate_included_in_beard_et_al,other_notes,risk_category
3,Will MacAskill,2019/2020,Existential risk in the 21st century,0.01,https://80000hours.org/podcast/episodes/will-m...,Yes,No,NaN,“Total risk” (or similar)
109,Brian Tomasik,2015/2018,"Chance that ""Humans will go extinct within mil...",0.05,https://reducing-suffering.org/summary-beliefs...,Yes,No,Discussed here: https://www.facebook.com/brian...,Miscellaneous
110,Pablo Stafforini,2015/2020,"Chance that ""Humans will go extinct within mil...",0.1,http://www.stafforini.com/blog/what_i_believe/,Yes,No,NaN,Miscellaneous


In [25]:
# Drop the rows that contain the risk categories
df_with_risk_category = df_with_risk_category[~df_with_risk_category['estimator'].isin(risk_categories)]
df_with_risk_category.head(3)

,estimator,date,estimation_measure,estimation,source,source_read_by_estimator,estimate_included_in_beard_et_al,other_notes,risk_category
1,Toby Ord,2020,“Total existential risk” by 2120,~17% (~1 in 6),The Precipice,Yes,No,"Ord writes: ""Don’t take these numbers to be co...",“Total risk” (or similar)
2,GCR Conference,2008,“Overall risk of extinction prior to 2100”,0.19,https://www.fhi.ox.ac.uk/reports/2008-1.pdf,Yes,Yes,This is the median. The report about these est...,“Total risk” (or similar)
3,Will MacAskill,2019/2020,Existential risk in the 21st century,0.01,https://80000hours.org/podcast/episodes/will-m...,Yes,No,NaN,“Total risk” (or similar)


In [26]:
# Rename risk categories to something more concise
df_with_risk_category['risk_category'] = df_with_risk_category['risk_category'].replace(risk_categories, risk_categories_aliases)
df_with_risk_category.sample(3)

,estimator,date,estimation_measure,estimation,source,source_read_by_estimator,estimate_included_in_beard_et_al,other_notes,risk_category
99,Wei Dai,2021,Expected fraction of total potential value tha...,>50%,Comment,Yes,No,"""What's your expectation of the fraction of to...",miscellaneous
72,Climate change,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nanotechnology
104,Toby Ord,2020,Existential catastrophe from “other anthropoge...,~2% (~1 in 50),The Precipice,Yes,No,See this post for some commentary: https://for...,miscellaneous


In [27]:
# Use only a few columns relevant for the analysis.
relevant_columns = [
    "estimator",
    "estimation_measure",
    "date",
    "estimation",
    "source_read_by_estimator",
    "risk_category",
    "other_notes"
]
df_relevant = df_with_risk_category[relevant_columns]
df_relevant.head(3)

,estimator,estimation_measure,date,estimation,source_read_by_estimator,risk_category,other_notes
1,Toby Ord,“Total existential risk” by 2120,2020,~17% (~1 in 6),Yes,total,"Ord writes: ""Don’t take these numbers to be co..."
2,GCR Conference,“Overall risk of extinction prior to 2100”,2008,0.19,Yes,total,This is the median. The report about these est...
3,Will MacAskill,Existential risk in the 21st century,2019/2020,0.01,Yes,total,NaN


In [28]:
# Save the dataframe to a CSV file
df_relevant.to_csv('./data/pre-processed_data.csv', index=False)

## Taking out the big guns

This might be total overkill for our purposes, but I find it fun, so, we'll use an LLM to help us process the rest of the columns.




### Estimation measure and estimation

The "estimation measure" varies slightly from estimation to estimation. It will be necessary to harmonize this into something more usable, like current existential risk per century.

For this, I'll attempt using an LLM to help us with this "translation".



In [29]:
# Initialize processor
from utils import RiskEstimateProcessor

processor = RiskEstimateProcessor("./models/llama-2-7b.Q4_K_M.gguf")

llama_load_model_from_file: using device Metal (Apple M1 Pro) - 10916 MiB free
llama_model_loader: loaded meta data with 19 key-value pairs and 291 tensors from ./models/llama-2-7b.Q4_K_M.gguf (version GGUF V2)
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = LLaMA v2
llama_model_loader: - kv   2:                       llama.context_length u32              = 4096
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 11008
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 128
llama_mod

In [30]:
# Load data
df = pd.read_csv('./data/pre-processed_data.csv')
df.sample(10)

,estimator,estimation_measure,date,estimation,source_read_by_estimator,risk_category,other_notes
64,"Anders Sandberg, adapting Denkenberger’s model",“Reduction in far future potential per year fr...,2018,~0.051%,Sort-of,nanotechnology,NaN
107,Paul Christiano,Amount by which “really nailing” some portion ...,2019,0.05,Yes,miscellaneous,"He says ""All of the numbers I’m going to give ..."
9,Wells,Annual probability as of 2009 of extinction,2009,0.3-0.4%,No,total,NaN
66,Pamlin & Armstrong,"""Infinite impact"" from nuclear war within the ...",2015,5e-05,No,nanotechnology,NaN
46,NaN,Aggregate estimate of biorisk's contribution t...,NaN,0.0002229877924,NaN,biorisk,NaN
5,Martin Rees,Odds that our present civilization on earth wi...,2003,"≤50% (""no better than fifty-fifty"")",No,total,NaN
56,NaN,NaN,NaN,NaN,NaN,nanotechnology,NaN
57,NaN,NaN,NaN,NaN,NaN,nanotechnology,NaN
108,Paul Christiano,Amount by which “a marginal person” doing some...,2019,"0.005% (“one in 20,000 or something”)",Yes,miscellaneous,"He says ""All of the numbers I’m going to give ..."
28,Ben Garfinkel,"""AI causing an existential catastrophe in the ...",2020,~0.1-1%,Yes,ai,Garfinkel was asked for his estimate during an...


In [31]:
# Now process the whole total risk dataframe
processed_df = processor.process_dataset(df)

llama_perf_context_print:        load time =    4344.99 ms
llama_perf_context_print: prompt eval time =       0.00 ms /   280 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /    89 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =   11561.71 ms /   369 tokens
Llama.generate: 37 prefix-match hit, remaining 184 prompt tokens to eval


Processed row 1 of 109


llama_perf_context_print:        load time =    4344.99 ms
llama_perf_context_print: prompt eval time =       0.00 ms /   184 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /    45 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =    8886.21 ms /   229 tokens
Llama.generate: 40 prefix-match hit, remaining 156 prompt tokens to eval


Processed row 2 of 109


llama_perf_context_print:        load time =    4344.99 ms
llama_perf_context_print: prompt eval time =       0.00 ms /   156 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /    54 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =    6990.90 ms /   210 tokens
Llama.generate: 37 prefix-match hit, remaining 162 prompt tokens to eval


...

Processed row 107 of 109


llama_perf_context_print:        load time =    4344.99 ms
llama_perf_context_print: prompt eval time =       0.00 ms /   199 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /    68 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =    7510.26 ms /   267 tokens
Llama.generate: 41 prefix-match hit, remaining 218 prompt tokens to eval


Processed row 108 of 109


llama_perf_context_print:        load time =    4344.99 ms
llama_perf_context_print: prompt eval time =       0.00 ms /   218 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /   252 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =   19289.00 ms /   470 tokens


Processed row 109 of 109


In [32]:
processed_df.sample(3)

,estimator,estimation_measure,date,estimation,source_read_by_estimator,risk_category,other_notes,century_probability,conversion_reasoning,conversion_confidence
48,NaN,NaN,NaN,NaN,NaN,nanotechnology,NaN,NaN,Error in processing: No JSON object found in r...,low
38,GCR Conference,Human extinction by 2100 as a result of “the s...,2008,0.0005,Yes,biorisk,This is the median. Beard et al.'s appendix sa...,0.0005,"The median estimate is 0.0005, which is the sa...",high
103,Brian Tomasik,"""The probability of civilizational collapse to...",2015,"""Maybe ~20%?""",Yes,miscellaneous,Tomasik was asked about his estimate of the ch...,0.2000,The probability of civilizational collapse to ...,high


In [33]:
# Save the processed data to a CSV file.
processed_df.to_csv('./data/risk_estimates_processed.csv', index=False)

In [34]:
# Validate results
validated_total_risk_df = processor.validate_estimates(processed_df)

# Basic analysis
summary = validated_total_risk_df.groupby('risk_category').agg({
    'century_probability': ['mean', 'median', 'std', 'count'],
    'conversion_confidence': lambda x: (x == 'high').mean()
})

Llama.generate: 3 prefix-match hit, remaining 121 prompt tokens to eval
llama_perf_context_print:        load time =    4344.99 ms
llama_perf_context_print: prompt eval time =       0.00 ms /   121 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /    63 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =    6395.70 ms /   184 tokens
Llama.generate: 20 prefix-match hit, remaining 55 prompt tokens to eval
llama_perf_context_print:        load time =    4344.99 ms
llama_perf_context_print: prompt eval time =       0.00 ms /    55 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /    63 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =    5752.40 ms /   118 tokens
Llama.generate: 23 prefix-match hit, remaining 61 prompt tokens to eval
llama_perf_con

In [35]:
processed_df.sample(10)

,estimator,estimation_measure,date,estimation,source_read_by_estimator,risk_category,other_notes,century_probability,conversion_reasoning,conversion_confidence,validation_notes
22,Rohin Shah,Probability of existential catastrophe due to ...,2020,Similar to 10%,Yes,ai,Rohin summarises the ecology / GPT perspective...,NaN,Error in processing: Requested tokens (537) ex...,low,"If you have any questions, please contact the ..."
49,Toby Ord,Existential catastrophe from “other anthropoge...,2020,~2% (~1 in 50),Yes,nanotechnology,See this post for some commentary: https://for...,0.001,The estimate is for a 2% chance of an existent...,high,"If you have any questions, please contact us a..."
91,Wei Dai,Expected fraction of total potential value tha...,2021,>50%,Yes,miscellaneous,"""What's your expectation of the fraction of to...",NaN,"Error in processing: Expecting ',' delimiter: ...",low,"If you have any questions, please contact the ..."
37,Toby Ord,Existential catastrophe from “engineered pande...,2020,~3% (~1 in 30),Yes,biorisk,NaN,0.003,nan,high,"If you have any questions, please contact the ..."
102,Pablo Stafforini,"Chance that ""Humans will go extinct within mil...",2015/2020,0.1,Yes,miscellaneous,NaN,NaN,Error in processing: No JSON object found in r...,low,"If the conversion is valid, please move on to ..."
23,Toby Ord,Existential catastrophe by 2120 as a result of...,2020,~10%,Yes,ai,Commentary here: https://forum.effectivealtrui...,0.100,The estimate is for a 10% chance of existentia...,high,"If you have any questions, please contact me a..."
2,Will MacAskill,Existential risk in the 21st century,2019/2020,0.01,Yes,total,NaN,0.010,Existential risk in the 21st century,high,"If you have any questions, please contact the ..."
30,Buck Shlegris,"""the probability of AI-induced existential ris...",2020,0.5,Yes,ai,Note that Buck gave a 25 percentage point lowe...,0.050,"The original estimate was 0.5, and the estimat...",high,"If you find an issue, please fix it and then s..."
88,Matthew Barnett,Tthe risk of human extinction due to SETI,2022,~0.1-0.2%,Yes,miscellaneous,"""I currently think there is roughly a 99% chan...",0.001,"The estimate is very low, and the reasoning is...",low,"If the output is ""invalid"", please describe th..."
27,Pamlin & Armstrong,"""Infinite impact"" from AI within the next 100 ...",2015,0-10%,No,ai,NaN,NaN,Error in processing: No JSON object found in r...,low,"If you have any questions, please contact the ..."


In [36]:
# Save the validated data to a CSV file.
validated_total_risk_df.to_csv('./data/risk_estimates_with_validation_remarks.csv', index=False)